## Simple XGBoost Model
### Using XGBoost

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O
import xgboost as xgb # XGBoost model
import matplotlib.pyplot as plt
import math

In [ ]:
def rmsle(pred, train):
	label = train.get_label()
	assert len(pred) == len(label)
	label = label.tolist()
	pred = pred.tolist()
	summ = [(math.log(label[i] + 1) - math.log(max(0, pred[i]) + 1)) ** 2.0 for i, j in enumerate(label)]
	return 'rmsle', (sum(summ) * (1.0 / len(pred))) ** 0.5

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_macro = pd.read_csv('../input/macro.csv')

In [ ]:
y_train = df_train['price_doc'].values

In [ ]:
df_train = df_train.drop('price_doc', axis=1)

In [ ]:
df_train = pd.merge(df_train, df_macro, how='left', on='timestamp')
df_test = pd.merge(df_test, df_macro, how='left', on='timestamp')

In [ ]:
print(df_train.dtypes.value_counts())
print(df_test.dtypes.value_counts())

In [ ]:
df_train['timestamp'] = pd.to_numeric(pd.to_datetime(df_train['timestamp'])) / 1e18
df_test['timestamp'] = pd.to_numeric(pd.to_datetime(df_test['timestamp'])) / 1e18
print(df_train['timestamp'].head())

In [ ]:
df_train = df_train.drop(['id'], axis=1)

In [ ]:
result = pd.DataFrame()

In [ ]:
result['id'] = df_test['id'].values
df_test = df_test.drop(['id'], axis=1)

In [ ]:
train_columns = list(
	set(df_train.select_dtypes(include=['float64', 'int64']).columns))

test_columns = list(
	set(df_test.select_dtypes(include=['float64', 'int64']).columns))

In [ ]:
x_train = df_train[train_columns].values
x_test = df_test[test_columns].values

In [ ]:
#from sklearn.model_selection import train_test_split
#x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train, test_size = 0.2, random_state = 64)

In [ ]:
split = 25000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

In [ ]:
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

In [ ]:
params = {}
params['objective'] = 'reg:linear' # linear regression
params['eta'] = 0.031              # step size shrinkage used in update to prevent overfitting
params['max_depth'] = 6            # maximum depth of a tree
params['silent'] = 1               # silent mode

In [ ]:
evals = [(d_train, 'train'), (d_valid, 'valid')]

In [ ]:
clf = xgb.train(params, d_train, 800, evals, feval=rmsle, early_stopping_rounds=100)

In [ ]:
x_test = xgb.DMatrix(x_test)
pred_test = clf.predict(x_test)

In [ ]:
result['price_doc'] = pred_test
result.to_csv('results.csv', index=False)

In [ ]:
Importing to CV
Overall kaggle score is ~0.324